# Traitement du calendrier

## Importation des librairies

In [21]:
import pandas as pd
import json as json
from langdetect import detect

In [22]:
def read_file(filename):
     return json.loads(open(filename, 'r').read())

def recup_data(data):
    return pd.DataFrame(data)

## Récupération des données

In [63]:
data = recup_data(read_file('data/calendar_etienne.txt'))
data.dropna(inplace = True)
data['lang_detect'] = data['description'].apply(detect)
data = data.loc[data.lang_detect == 'fr']
data.reset_index(inplace = True, drop = True)
data

,summary,description,location,startTime,endTime,attachements,lang_detect
0,Vol Paris-Singapour,Vol AF0256 Paris (CDG) - Singapour(SIN)\nHeure...,AÃ©roport Charles de Gaulle (CDG) - Terminal 2E,2016-12-17T21:00:00+01:00,2016-12-18T09:30:00+01:00,[],fr
1,Vol Singapour-Paris,Vol AF0257 Singapour (SIN) - Paris(CDG)\nHeure...,Changi Airport (SIN) - Terminal 1,2016-12-29T16:05:00+01:00,2016-12-30T06:10:00+01:00,[],fr
2,Vol Ã destination de Frankfurt (LH 1041),Pour afficher des informations dÃ©taillÃ©es su...,Paris CDG,2018-04-19T18:30:00+02:00,2018-04-19T19:40:00+02:00,[],fr
3,Vol Ã destination de Singapore (LH 778),Pour afficher des informations dÃ©taillÃ©es su...,Frankfurt FRA,2018-04-19T21:55:00+02:00,2018-04-20T10:15:00+02:00,[],fr
4,Vol Ã destination de Munich (LH 791),Pour afficher des informations dÃ©taillÃ©es su...,Singapore SIN,2018-04-27T16:55:00+02:00,2018-04-28T05:25:00+02:00,[],fr
...,...,...,...,...,...,...,...
83,TEAM UP,Pour afficher des informations dÃ©taillÃ©es su...,"CentraleSupÃ©lec - BÃ¢timent Eiffel, 8 Rue Jol...",2019-10-10T17:00:00+02:00,2019-10-10T22:00:00+02:00,[],fr
84,[PAYLEAD] Rencontre Etienne / Equipe Data,"Hello Etienne,\r\n\r\nComme discutÃ© au tÃ©l.....","69 Rue d'Hauteville, appelle moi au 06 03 02 0...",2019-11-05T15:00:00+01:00,2019-11-05T17:00:00+01:00,[],fr
85,Brunch Open source Vue.js / Nuxt.js,Pour afficher des informations dÃ©taillÃ©es su...,"Theodo, 48 Boulevard des Batignolles75017 Pari...",2019-11-30T11:30:00+01:00,2019-11-30T18:00:00+01:00,[],fr
86,Vol Ã destination de London (AF 1780),Pour afficher des informations dÃ©taillÃ©es su...,Paris CDG,2019-12-08T13:20:00+01:00,2019-12-08T14:45:00+01:00,[],fr


## Modèle LDA

### Imports for LDA

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [65]:
columns_lda = ['summary', 'description', 'location']
data_lda = data[columns_lda].copy()
data_lda.reset_index(inplace = True, drop = True)

In [66]:
data_lda['agg_content'] = ['']*data_lda.shape[0]
for el in columns_lda:
    data_lda.agg_content = data_lda.agg_content + ['']*data_lda.shape[0] + data_lda[el]

In [67]:
vect = TfidfVectorizer() #use_idf = False
X = vect.fit_transform(data_lda.agg_content)

In [68]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [69]:
for i in range (1,10):
    print('### %i Topics ###' %i)
    lda = LatentDirichletAllocation(n_components = i)
    lda.fit(X)
    no_top_words = 10
    tfidf_feature_names = vect.get_feature_names()
    display_topics(lda, tfidf_feature_names, no_top_words)
    print("\n\n")

### 1 Topics ###
Topic 0:
cours groupe langues de 24 places info intervenants salles mail



### 2 Topics ###
Topic 0:
langues 24 de cours groupe mail places intervenants les salles
Topic 1:
info 60 salle magistral en dirigã travaux places groupe amphi



### 3 Topics ###
Topic 0:
mail que crã vã https tã sur 40 es nements
Topic 1:
info salle en travaux dirigã groupe 20 18 1218 placesinfo
Topic 2:
cours langues 24 de groupe places tudiants tous 60 intervenants



### 4 Topics ###
Topic 0:
module 2a 32 contrã systã teyssandier me nierie nicolas ingã
Topic 1:
info 60 salle magistral en travaux dirigã groupe places amphi
Topic 2:
langues 24 cours de groupe tudiants tous places places1226 allemand
Topic 3:
mail que vã crã https tã sur dã un cal



### 5 Topics ###
Topic 0:
contrã le exploitation in201 systã mes 1151 du olivier nicole
Topic 1:
mail crã https vã tã que info sur salle de
Topic 2:
32 culture thã enseignement matique module 2a des techniques places1413
Topic 3:
cours langues 24

## Traitements des vols

In [78]:
columns_vols = ['summary', 'description', 'location']
data_vols = data.loc[data['summary'].str.contains('Vol')].copy() #Singapour|Florence|Vol

,summary,description,location,startTime,endTime,attachements,lang_detect
0,Vol Paris-Singapour,Vol AF0256 Paris (CDG) - Singapour(SIN)\nHeure...,AÃ©roport Charles de Gaulle (CDG) - Terminal 2E,2016-12-17T21:00:00+01:00,2016-12-18T09:30:00+01:00,[],fr
1,Vol Singapour-Paris,Vol AF0257 Singapour (SIN) - Paris(CDG)\nHeure...,Changi Airport (SIN) - Terminal 1,2016-12-29T16:05:00+01:00,2016-12-30T06:10:00+01:00,[],fr
2,Vol Ã destination de Frankfurt (LH 1041),Pour afficher des informations dÃ©taillÃ©es su...,Paris CDG,2018-04-19T18:30:00+02:00,2018-04-19T19:40:00+02:00,[],fr
3,Vol Ã destination de Singapore (LH 778),Pour afficher des informations dÃ©taillÃ©es su...,Frankfurt FRA,2018-04-19T21:55:00+02:00,2018-04-20T10:15:00+02:00,[],fr
4,Vol Ã destination de Munich (LH 791),Pour afficher des informations dÃ©taillÃ©es su...,Singapore SIN,2018-04-27T16:55:00+02:00,2018-04-28T05:25:00+02:00,[],fr
5,Vol Ã destination de Paris (LH 2226),Pour afficher des informations dÃ©taillÃ©es su...,Munich MUC,2018-04-28T06:35:00+02:00,2018-04-28T08:05:00+02:00,[],fr
6,Vol Ã destination de Francfort-sur-le-Main (L...,Pour afficher des informations dÃ©taillÃ©es su...,Lyon LYS,2018-10-29T13:30:00+01:00,2018-10-29T14:50:00+01:00,[],fr
7,Vol Ã destination de Florence (LH 316),Pour afficher des informations dÃ©taillÃ©es su...,Francfort-sur-le-Main FRA,2018-10-29T16:35:00+01:00,2018-10-29T18:00:00+01:00,[],fr
8,Vol Ã destination de Francfort-sur-le-Main (L...,Pour afficher des informations dÃ©taillÃ©es su...,Florence FLR,2018-11-02T18:40:00+01:00,2018-11-02T20:20:00+01:00,[],fr
9,Vol Ã destination de Lyon (LH 1082),Pour afficher des informations dÃ©taillÃ©es su...,Francfort-sur-le-Main FRA,2018-11-02T21:45:00+01:00,2018-11-02T23:00:00+01:00,[],fr


In [80]:
data.summary.unique()

array(['Vol Paris-Singapour', 'Vol Singapour-Paris',
       'Vol Ã\xa0 destination de Frankfurt (LH 1041)',
       'Vol Ã\xa0 destination de Singapore (LH 778)',
       'Vol Ã\xa0 destination de Munich (LH 791)',
       'Vol Ã\xa0 destination de Paris (LH 2226)',
       'Vol Ã\xa0 destination de Francfort-sur-le-Main (LH 1081)',
       'Vol Ã\xa0 destination de Florence (LH 316)',
       'Vol Ã\xa0 destination de Francfort-sur-le-Main (LH 317)',
       'Vol Ã\xa0 destination de Lyon (LH 1082)',
       'ES201 (Travaux dirigÃ©s en salle info)',
       'AN223 (Cours de Langues)', 'AN221 (Cours de Langues)',
       'ES201 (ContrÃ´le)', 'Remise des diplÃ´mes',
       'JournÃ©e entrepreneuriat Ã©tudiant PEIPS',
       'Vol Ã\xa0 destination de Berlin (U2 4634)',
       'Vol Ã\xa0 destination de BÃ¢le (U2 4641)',
       'Vol Ã\xa0 destination de BÃ¢le (U2 4637)',
       'IN201 (Cours magistral)',
       'IN201 (Travaux dirigÃ©s en salle info)',
       'SPBASK_S1 (Cours de sport)', 'SYS2 (Modu

# Brouillon

In [5]:
data.loc[data['summary'].str.contains('Vol')]

,summary,description,location,startTime,endTime,attachements,lang_detect
165,Vol Ã destination de Paris (TO 4031),Pour afficher des informations dÃ©taillÃ©es su...,Prague PRG,2017-11-18T09:15:00+01:00,2017-11-18T11:10:00+01:00,[],fr
166,Vol Ã destination de Prague (TO 4038),Pour afficher des informations dÃ©taillÃ©es su...,Paris (Orly Sud) ORY,2017-11-10T18:25:00+01:00,2017-11-10T20:10:00+01:00,[],fr
167,Vol Ã destination de Copenhagen (VY 8634),Pour afficher des informations dÃ©taillÃ©es su...,Paris ORY,2018-05-11T13:20:00+02:00,2018-05-11T15:15:00+02:00,[],fr
168,Vol Ã destination de Copenhague (U2 3827),Pour afficher des informations dÃ©taillÃ©es su...,Paris CDG,2018-06-24T18:30:00+02:00,2018-06-24T20:30:00+02:00,[],fr
169,Vol Ã destination de Nice (D8 3644),Pour afficher des informations dÃ©taillÃ©es su...,Copenhague CPH,2018-08-04T18:40:00+02:00,2018-08-04T21:00:00+02:00,[],fr
171,VolÂ : U2 5537 entre TXL et ORY,Code de confirmationÂ : EW3J5FL\n\nM PAUL AZOU...,AÃ©roport de Berlin-Tegel\nBerlin Tegel Airpor...,2018-11-08T20:30:00+01:00,2018-11-08T22:20:00+01:00,[],fr
182,Vol Ã destination de Berlin (U2 5538),Pour afficher des informations dÃ©taillÃ©es su...,Paris ORY,2018-11-12T18:15:00+01:00,2018-11-12T20:00:00+01:00,[],fr
183,Vol Ã destination de Paris (U2 5537),Pour afficher des informations dÃ©taillÃ©es su...,Berlin TXL,2018-12-21T20:30:00+01:00,2018-12-21T22:20:00+01:00,[],fr
184,Vol Ã destination de Berlin (U2 5538),Pour afficher des informations dÃ©taillÃ©es su...,Paris ORY,2019-01-02T18:15:00+01:00,2019-01-02T20:00:00+01:00,[],fr
186,Vol Ã destination de Paris (U2 5675),Pour afficher des informations dÃ©taillÃ©es su...,Berlin TXL,2019-03-21T20:30:00+01:00,2019-03-21T22:20:00+01:00,[],fr


In [6]:
data['description'].unique()

array(['Modules 3A\n\nCryptographie : principes et mise en oeuvre\nExamen\nGroupe : Tous les Ã©tudiants\nIntervenants : BYRON Quentin\nSalles : 1225 - 24 places',
       'Modules 3A en salle info\n\nSystÃ¨mes de gestion de bases de donnÃ©es\nGroupe : Tous les Ã©tudiants\nIntervenants : ANCIAUX Nicolas\nSalles : INFO 1151 - 18 places',
       'Modules 3A en salle info\n\nUnix avancÃ© et scripts\nGroupe : Tous les Ã©tudiants\nIntervenants : JOIRET Denis\nSalles : INFO 1149 - 20 places',
       "Modules 3A en salle info\n\nArchitecture des systÃ¨mes d'information\nGroupe : Tous les Ã©tudiants\nSalles : INFO 1151 - 18 places",
       'Modules 3A en salle info\n\nAnalyse de vulnÃ©rabilitÃ© des sources et binaires\nGroupe : Tous les Ã©tudiants\nIntervenants : BONICHON Richard\nSalles : INFO 1151 - 18 places',
       'Modules 3A\n\nGestion des ressources humaines et management interculturel\nGroupe : Tous les Ã©tudiants\nIntervenants : LALLOUANI Amira, MOUSSAOUI Nabil\nSalles : Amphi 200 plac

In [7]:
data['lang_detect'] = data['description'].apply(detect)

C:\Users\thepl\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
data.lang_detect.unique()

array(['fr', 'en'], dtype=object)

In [9]:
data.loc[data.summary.str.contains('Module')]

,summary,description,location,startTime,endTime,attachements,lang_detect
0,ASI331 (Modules 3A),Modules 3A\n\nCryptographie : principes et mis...,1225 - 24 places,2019-10-25T11:30:00Z,2019-10-25T13:30:00Z,[],fr
1,ASI313 (Modules 3A en salle info),Modules 3A en salle info\n\nSystÃ¨mes de gesti...,INFO 1151 - 18 places,2020-01-20T07:30:00Z,2020-01-20T11:15:00Z,[],fr
2,ASI313 (Modules 3A en salle info),Modules 3A en salle info\n\nSystÃ¨mes de gesti...,INFO 1151 - 18 places,2020-01-13T07:30:00Z,2020-01-13T11:15:00Z,[],fr
3,ASI315 (Modules 3A en salle info),Modules 3A en salle info\n\nUnix avancÃ© et sc...,INFO 1149 - 20 places,2020-01-27T07:30:00Z,2020-01-27T11:15:00Z,[],fr
4,ASI315 (Modules 3A en salle info),Modules 3A en salle info\n\nUnix avancÃ© et sc...,INFO 1149 - 20 places,2020-01-06T07:30:00Z,2020-01-06T11:15:00Z,[],fr
...,...,...,...,...,...,...,...
132,ASI331 (Modules 3A en salle info),Modules 3A en salle info\n\nCryptographie : pr...,INFO 1151 - 18 places,2019-10-14T13:45:00Z,2019-10-14T15:45:00Z,[],fr
155,ASI331 (Modules 3A),Modules 3A\n\nCryptographie : principes et mis...,1312 - 24 places,2019-10-04T11:30:00Z,2019-10-04T13:00:00Z,[],fr
156,ASI311 (Modules 3A en salle info),Modules 3A en salle info\n\nPlateformes logici...,1426 - 40 places,2019-10-04T07:00:00Z,2019-10-04T10:45:00Z,[],fr
157,ASI311 (Modules 3A en salle info),Modules 3A en salle info\n\nPlateformes logici...,1426 - 40 places,2019-10-11T07:00:00Z,2019-10-11T10:45:00Z,[],fr


In [10]:
data.loc[data.description.str.contains('entretien')]

,summary,description,location,startTime,endTime,attachements,lang_detect
137,KITE (Petite classe),Petite classe\n\nKITE\ncours deburf entretiens...,1426 - 40 places,2019-11-26T13:30:00Z,2019-11-26T16:30:00Z,[],fr
179,Entretien pour le poste de Stage - Developpeur...,"Â \n\n Bonjour,\n\n\n\nSujet: Entretien pour l...",LES DUNES - Val de Fontenay,2018-12-18T14:00:00+01:00,2018-12-18T15:00:00+01:00,[],fr
